Define your variables (That's the only cells you need to change). Then run the whole notebook

In [1]:
method = 'SyNRA'
metric = 'mattes'
path_to_data = '/home/tli_project/Desktop/Data/T4/stage1/211025/'
save_path = '/home/tli_project/Desktop/Data/T4/stage1/211025/'
group_by = '_phase.tif'
ch_names = ['GFP', 'Red']
save_name = 'Rigid_211025_P36.5'
compile = False
filter = False

In [2]:
if save_path[-1] != '/':
    save_path += '/'

Load packages and define functions

In [3]:
import os
from timeit import default_timer as timer

In [4]:
import utils.datautils as datautils
import utils.registration as registration

/home/tli_project/.local/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


Running the pipeline

In [5]:
# reading list of 3D images into dictionary of channels as indicated in ch_names, and applying median filter on the last ch
image_4D = datautils.read_files(path_to_data, group_by ,compile=compile, ch_names=ch_names, filter=filter,
                                order=False, save=True, save_path=save_path, save_file=group_by+'_raw', 
                                xy_pixel=0.076, z_pixel=0.4)

first 5 files
['/home/tli_project/Desktop/Data/T4/stage1/211025/Rigid_211025_P36.5_GFP_phase.tif', '/home/tli_project/Desktop/Data/T4/stage1/211025/Rigid_211025_P36.5_Red_phase.tif']


In [7]:
# # in case for manual shift
# from skimage.filters import gaussian
# from skimage.registration import phase_cross_correlation as corr
# from tqdm import tqdm
# import pandas as pd
# from scipy import ndimage
# import numpy as np

# shift = np.array([10,0,-16])
# for ch, img in image_4D.items():
#     for ind, stack in tqdm(enumerate(img[90:], start=90)):
#         temp = ndimage.shift(stack, shift) 
#         image_4D[ch][ind] = temp.copy()

1it [00:22, 22.24s/it]
1it [00:20, 20.68s/it]


In [8]:
# save_name = 'Rigid_221015_P36_stitched3'
# import tifffile as tif
# for ch, img in image_4D.items():
#     name = save_path+save_name+'_'+ch+'.tif'
#     tif.imwrite(name, img, imagej=True, resolution=(1./0.076, 1./0.076),
#                 metadata={'spacing': 0.4, 'unit': 'um', 'finterval': 1/10,'axes': 'TZYX'})

/home/tli_project/.local/lib/python3.8/site-packages/tifffile/tifffile.py:2876: UserWarning:

<tifffile.TiffWriter 'Rigid_221015_P36…itched3_GFP.tif'> truncating ImageJ file

/home/tli_project/.local/lib/python3.8/site-packages/tifffile/tifffile.py:2876: UserWarning:

<tifffile.TiffWriter 'Rigid_221015_P36…itched3_Red.tif'> truncating ImageJ file



In [6]:
# measuring the cosine_similarity check of GFP channel before registration
pre_checks = datautils.similarity_4D(image_4D[ch_names[0]], 
                                    save=True, 
                                    save_path=save_path, 
                                    save_file='pre_checks.csv')

In [6]:
# run this cell if you want to apply phase_correlation
from skimage.filters import gaussian
from skimage.registration import phase_cross_correlation as corr
from tqdm import tqdm
import pandas as pd
from scipy import ndimage
import numpy as np
import tifffile as tif

shifts = []
for ind, st in tqdm(enumerate(image_4D['GFP'][1:])):
    fixed = image_4D['GFP'][ind].copy()
    moving = st.copy()
    fixed = gaussian(fixed, sigma=1)
    moving = gaussian(moving, sigma=1)
    shift, error, diffphase = corr(fixed, moving)
    shifts.append(shift)

shifts_s = pd.Series(shifts)
name = save_path+save_name+'Phase_shifts.csv'
shifts_s.to_csv(name)

current_shift = np.array([0,0,0])
for ind, shi in enumerate(shifts):
    current_shift = current_shift+shi
    image_4D['GFP'][ind+1] = ndimage.shift(image_4D['GFP'][ind+1], current_shift) 
    image_4D['Red'][ind+1] = ndimage.shift(image_4D['Red'][ind+1], current_shift)

for ch, img in image_4D.items():
    name = save_path+save_name+'_'+ch+'_phase.tif'
    tif.imwrite(name, img, imagej=True, resolution=(1./0.076, 1./0.076),
                metadata={'spacing': 0.4, 'unit': 'um', 'finterval': 1/10,'axes': 'TZYX'})

85it [14:45, 10.42s/it]
/home/tli_project/.local/lib/python3.8/site-packages/tifffile/tifffile.py:2876: UserWarning:

<tifffile.TiffWriter 'Rigid_211025_P36.5_GFP_phase.tif'> truncating ImageJ file

/home/tli_project/.local/lib/python3.8/site-packages/tifffile/tifffile.py:2876: UserWarning:

<tifffile.TiffWriter 'Rigid_211025_P36.5_Red_phase.tif'> truncating ImageJ file



In [7]:
# running antspy, saving output into temp 3D files, and then compiling them to 4D and finally deleting the temp 3D files
# name_shifts = method+csv_name
registration.antspy_drift_corr(img_4D_g=image_4D[ch_names[0]],
                                img_4D_r=image_4D[ch_names[1]], 
                                ch_names=ch_names,
                                save_path=save_path, 
                                save_name=save_name,
                                ref_t = 7,
                                # ref_t = int(len(image_4D[ch_names[0]])/2),
                                drift_corr=method, metric=metric) 

applying_antspy: 100%|██████████| 87/87 [7:02:16<00:00, 291.22s/it]  


In [8]:
del image_4D

In [9]:
# compiling the registered 3D images into 4D and deleting the temp files for both channels 
gfp_name = ch_names[0]+'_'+method+'_4D_'+save_name+'.tif'
gfp_list = datautils.get_file_names(path=save_path, 
                                    group_by=ch_names[0]+'_'+method, 
                                    order=False, nested_files=False, criteria='tif')
gfp_4D = datautils.files_to_4D(files_list=gfp_list, ch_names=[ch_names[0]], filter=False, 
                                save=True, save_path=save_path, save_file=gfp_name,
                                xy_pixel=0.0764616, z_pixel=0.4)
for file in gfp_list:
    os.remove(file)

# measuring the cosine_similarity check of GFP channel after registration
post_checks = datautils.similarity_4D(gfp_4D[ch_names[0]], save=True, 
                                        save_path=save_path, 
                                        save_file=method+'_checks.csv')

del gfp_4D

red_name = ch_names[1]+'_'+method+'_4D_'+save_name+'.tif'
red_list = datautils.get_file_names(path=save_path, 
                                    group_by=ch_names[1]+'_'+method, 
                                    order=False, 
                                    nested_files=False, criteria='tif')
red_4D = datautils.files_to_4D(files_list=red_list, ch_names=[ch_names[1]], filter=False,
                                save=True, save_path=save_path, save_file=red_name,
                                xy_pixel=0.0764616, z_pixel=0.4)
for file in red_list:
    os.remove(file)

# deleting the final compiled 4D images
del red_4D 

first 5 files
['/home/tli_project/Desktop/Data/T4/stage1/211025/GFP_SyNRA_Rigid_211025_P36.5_001.tif', '/home/tli_project/Desktop/Data/T4/stage1/211025/GFP_SyNRA_Rigid_211025_P36.5_002.tif', '/home/tli_project/Desktop/Data/T4/stage1/211025/GFP_SyNRA_Rigid_211025_P36.5_003.tif', '/home/tli_project/Desktop/Data/T4/stage1/211025/GFP_SyNRA_Rigid_211025_P36.5_004.tif', '/home/tli_project/Desktop/Data/T4/stage1/211025/GFP_SyNRA_Rigid_211025_P36.5_005.tif']


dict_keys(['GFP']) <class 'list'> 86
compiling the GFP channel


/home/tli_project/.local/lib/python3.8/site-packages/tifffile/tifffile.py:2876: UserWarning:

<tifffile.TiffWriter '4D_GFP_GFP_SyNRA…11025_P36.5.tif'> truncating ImageJ file



first 5 files
['/home/tli_project/Desktop/Data/T4/stage1/211025/Red_SyNRA_Rigid_211025_P36.5_001.tif', '/home/tli_project/Desktop/Data/T4/stage1/211025/Red_SyNRA_Rigid_211025_P36.5_002.tif', '/home/tli_project/Desktop/Data/T4/stage1/211025/Red_SyNRA_Rigid_211025_P36.5_003.tif', '/home/tli_project/Desktop/Data/T4/stage1/211025/Red_SyNRA_Rigid_211025_P36.5_004.tif', '/home/tli_project/Desktop/Data/T4/stage1/211025/Red_SyNRA_Rigid_211025_P36.5_005.tif']


dict_keys(['Red']) <class 'list'> 86
compiling the Red channel


/home/tli_project/.local/lib/python3.8/site-packages/tifffile/tifffile.py:2876: UserWarning:

<tifffile.TiffWriter '4D_Red_Red_SyNRA…11025_P36.5.tif'> truncating ImageJ file



In [ ]:
print('memory cleared, and kernel stopped')
print('Job is DONE!')
os._exit(00)

Error: Canceled future for execute_request message before replies were done